# Temporal Score Rescaling Code Demo


This notebook demonstrates the usage of TSR schedulers. While we show simple examples with Stable Diffusion 3 and Stable Diffusion 2, the scheduler classes are compatible to any diffusion and flow models implemented in the diffusers framework.

Requires installing Diffusers before running.

In [ ]:
import torch
from diffusers import StableDiffusion3Pipeline, StableDiffusionPipeline
from diffusers.schedulers import FlowMatchEulerDiscreteScheduler, DDIMScheduler
from TSR_diffusers.TSR_schedulers import TSR_FlowEulerScheduler, TSR_DDIMScheduler

device = torch.device("cuda")
model_id = "stabilityai/stable-diffusion-3-medium-diffusers"

orig_scheduler = FlowMatchEulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
scheduler = TSR_FlowEulerScheduler.from_config(orig_scheduler.config, k=1.0, psr_sigma=1.0, orig_scheduler=orig_scheduler)
pipe = StableDiffusion3Pipeline.from_pretrained(
    model_id,
    text_encoder_3=None, # Disabling T5 for efficiency
    tokenizer_3=None,
    torch_dtype=torch.float16,    
).to(device)
pipe.scheduler = scheduler



In [ ]:
torch.manual_seed(42)
init_noises = torch.randn([1, 16, 128, 128], device=device, dtype=torch.float16)

prompt = "A photo of a corgi"
negative_prompt = ""

pipe.scheduler.k = 0.95 # explore different values!
pipe.scheduler.psr_sigma = 1.0

with torch.no_grad():
    image = pipe(
        latents=init_noises,
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=30,
        num_images_per_prompt=1,
        height=1024,
        width=1024,
        guidance_scale=7.5,
    ).images

image[0]

In [ ]:
repo_id = "stabilityai/stable-diffusion-2-base"
orig_scheduler = DDIMScheduler.from_pretrained(repo_id, subfolder="scheduler")
scheduler = TSR_DDIMScheduler.from_config(orig_scheduler.config, k=1.0, psr_sigma=1.0, orig_scheduler=orig_scheduler)
pipe = StableDiffusionPipeline.from_pretrained(
    repo_id,
    torch_dtype=torch.float16,    
).to(device)
pipe.scheduler = scheduler


In [ ]:
torch.manual_seed(0)
init_noises = torch.randn([1,4,64,64], device=device, dtype=pipe.dtype)

prompt = "A photo of a mountain under the starry night"
negative_prompt = ""

pipe.scheduler.k = 0.95 # explore different values!
pipe.scheduler.psr_sigma = 1.0

with torch.no_grad():
    image = pipe(
        latents=init_noises,
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=50,
        num_images_per_prompt=1,
        guidance_scale=7.5,
    ).images

image[0]